In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys
sys.path.append("../")
from sklearn.metrics import roc_auc_score
from src.utils import get_activations
from src.data.isic import build_isic_dp

/home/ksaab/miniconda3/envs/mlhc/lib/python3.9/site-packages/meerkat/nn/__init__.py:7: ExperimentalWarning: The `meerkat.nn` module is experimental and has limited test coverage. Proceed with caution.
  warnings.warn(


In [3]:
# build datapanel
dp = build_isic_dp()
len(dp)

2853

### Evaluation on ERM model

In [6]:
segmentation = False
model_pth = "/home/ksaab/Documents/spatial_specificity/results/dataset_isic/method_erm/lr_0.0005/wd_0.01/dropout_0/seed_1/domino/2uruxdkm/checkpoints/epoch=2-step=230.ckpt"

test_mask = np.array(dp["split"].data) == "test"
test_dp = dp.lz[test_mask]
print(len(test_dp))
test_dp_ = get_activations(test_dp, model_pth, segmentation=segmentation, isic=True, batch_size=4)

# for segmentation, we have 3 labels, one for background, one for benign and the last for malignant
test_probs = test_dp_["output"][:,2] if segmentation else test_dp_["output"][:,1]
test_labels = test_dp_["target"][:]
overall_score = roc_auc_score(test_labels,test_probs)

print(f"Overall/Robust AUROC: {overall_score:.4f}")

772


  0%|          | 0/193 [00:00<?, ?it/s]

Overall/Robust AUROC: 0.3098


### Evaluation on segmentation model

In [12]:
segmentation = True
model_pth = "/home/ksaab/Documents/spatial_specificity/results/method_seg/lr_0.0005/wd_0/dropout_0/seed_1/domino/maf4yoer/checkpoints/epoch=1-step=115.ckpt" 

test_dp_ = get_activations(test_dp, model_pth, segmentation=segmentation, batch_size=4, isic=True)

test_probs = test_dp_["output"][:,2] if segmentation else test_dp_["output"][:,1]
test_labels = test_dp_["target"][:]
overall_score = roc_auc_score(test_labels,test_probs)

print(f"Overall/Robust AUROC: {overall_score:.4f}")

  0%|          | 0/193 [00:00<?, ?it/s]

Overall/Robust AUROC: 0.7724
